In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [21]:
def load_dataset():
    train_dataset = h5py.File('../train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('../test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [22]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

In [23]:
X_train_orig.shape

(1080, 64, 64, 3)

In [24]:
Y_train_orig.shape

(1, 1080)

In [25]:
X_train=X_train_orig
Y_train=Y_train_orig.T

In [26]:
print (X_train.shape)
print (Y_train.shape)

(1080, 64, 64, 3)
(1080, 1)


In [27]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [28]:
Y_train=convert_to_one_hot(Y_train, 6)
print (Y_train.shape)


(6, 1080)


In [29]:
Y_train=Y_train.T
print (Y_train.shape)
X_train = X_train_orig/255


(1080, 6)


In [30]:
X_test=X_test_orig/255
print(X_test.shape)
print (Y_test_orig.shape)

(120, 64, 64, 3)
(1, 120)


In [31]:
Y_test=convert_to_one_hot(Y_test_orig,6)
Y_test=Y_test.T
print (Y_test.shape)

(120, 6)


In [32]:
tf.reset_default_graph()
x=tf.placeholder(tf.float32, shape=[None, 64,64,3])
y=tf.placeholder(tf.float32, shape=[None, 6])

In [33]:
W1=tf.get_variable("W1",shape=[4,4,3,8],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())
W2=tf.get_variable("W2",shape=[2,2,8,16],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())

In [34]:
z1=tf.nn.conv2d(x,W1,strides=[1,1,1,1],padding='SAME')
a1=tf.nn.relu(z1)
p1=tf.nn.max_pool(a1,ksize=[1,8,8,1],strides=[1,8,8,1],padding='SAME')
z2=tf.nn.conv2d(p1,W2,strides=[1,1,1,1],padding='SAME')
a2=tf.nn.relu(z2)
p2=tf.nn.max_pool(a2,ksize=[1,4,4,1],strides=[1,4,4,1],padding='SAME')

p=tf.contrib.layers.flatten(p2)
z3=tf.contrib.layers.fully_connected(p,6,activation_fn=None)


In [35]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( logits=z3, labels=y))
optimizer=tf.train.AdamOptimizer(0.009).minimize(cost)

In [36]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    
    sess.run(init)
    cos=0
    for epoch in range(300):
        _,cost_=sess.run([optimizer,cost],feed_dict={x:X_train, y:Y_train})
        cos=cost_
        print ("Cost after epoch %i: %f" % (epoch, cos))
        
    predict_op=tf.argmax(z3,1)
    #print(predict_op.shape)
    #print (tf.argmax(y, 1).shape)
    correct_prediction = tf.equal(predict_op, tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    train_accuracy = accuracy.eval({x: X_train, y: Y_train})
    test_accuracy = accuracy.eval({x: X_test, y: Y_test})
    print(train_accuracy)
    print (test_accuracy)

Cost after epoch 0: 1.863419
Cost after epoch 1: 1.789062
Cost after epoch 2: 1.796920
Cost after epoch 3: 1.794109
Cost after epoch 4: 1.790303
Cost after epoch 5: 1.789416
Cost after epoch 6: 1.790927
Cost after epoch 7: 1.790393
Cost after epoch 8: 1.788706
Cost after epoch 9: 1.786367
Cost after epoch 10: 1.784553
Cost after epoch 11: 1.783780
Cost after epoch 12: 1.781944
Cost after epoch 13: 1.779004
Cost after epoch 14: 1.775996
Cost after epoch 15: 1.773175
Cost after epoch 16: 1.769244
Cost after epoch 17: 1.764660
Cost after epoch 18: 1.758409
Cost after epoch 19: 1.749828
Cost after epoch 20: 1.741842
Cost after epoch 21: 1.730837
Cost after epoch 22: 1.720982
Cost after epoch 23: 1.707078
Cost after epoch 24: 1.698973
Cost after epoch 25: 1.679195
Cost after epoch 26: 1.665466
Cost after epoch 27: 1.642042
Cost after epoch 28: 1.634576
Cost after epoch 29: 1.605117
Cost after epoch 30: 1.592451
Cost after epoch 31: 1.558734
Cost after epoch 32: 1.547498
Cost after epoch 33:

Cost after epoch 268: 0.131563
Cost after epoch 269: 0.130265
Cost after epoch 270: 0.129264
Cost after epoch 271: 0.127059
Cost after epoch 272: 0.128322
Cost after epoch 273: 0.126496
Cost after epoch 274: 0.125459
Cost after epoch 275: 0.125355
Cost after epoch 276: 0.122439
Cost after epoch 277: 0.123807
Cost after epoch 278: 0.122515
Cost after epoch 279: 0.121144
Cost after epoch 280: 0.121465
Cost after epoch 281: 0.118950
Cost after epoch 282: 0.118935
Cost after epoch 283: 0.118384
Cost after epoch 284: 0.117154
Cost after epoch 285: 0.117076
Cost after epoch 286: 0.115728
Cost after epoch 287: 0.114986
Cost after epoch 288: 0.114308
Cost after epoch 289: 0.113579
Cost after epoch 290: 0.113056
Cost after epoch 291: 0.112284
Cost after epoch 292: 0.111663
Cost after epoch 293: 0.110726
Cost after epoch 294: 0.110075
Cost after epoch 295: 0.109458
Cost after epoch 296: 0.108615
Cost after epoch 297: 0.108278
Cost after epoch 298: 0.107612
Cost after epoch 299: 0.107004
0.982407

In [37]:
print (train_accuracy)

0.9824074


In [39]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    
    sess.run(init)
    cos=0
    for epoch in range(100):
        _,cost_=sess.run([optimizer,cost],feed_dict={x:X_train, y:Y_train})
        cos=cost_
        print ("Cost after epoch %i: %f" % (epoch, cos))
        
    predict_op=tf.argmax(z3,1)
    #print(predict_op.shape)
    #print (tf.argmax(y, 1).shape)
    correct_prediction = tf.equal(predict_op, tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    train_accuracy = accuracy.eval({x: X_train, y: Y_train})
    test_accuracy = accuracy.eval({x: X_test, y: Y_test})
    print(train_accuracy)
    print (test_accuracy)

Cost after epoch 0: 1.946557
Cost after epoch 1: 1.811894
Cost after epoch 2: 1.802021
Cost after epoch 3: 1.803209
Cost after epoch 4: 1.796831
Cost after epoch 5: 1.790282
Cost after epoch 6: 1.788901
Cost after epoch 7: 1.788493
Cost after epoch 8: 1.787221
Cost after epoch 9: 1.785748
Cost after epoch 10: 1.784229
Cost after epoch 11: 1.782375
Cost after epoch 12: 1.780360
Cost after epoch 13: 1.778495
Cost after epoch 14: 1.775382
Cost after epoch 15: 1.771788
Cost after epoch 16: 1.767779
Cost after epoch 17: 1.762056
Cost after epoch 18: 1.756683
Cost after epoch 19: 1.749743
Cost after epoch 20: 1.739931
Cost after epoch 21: 1.730112
Cost after epoch 22: 1.720192
Cost after epoch 23: 1.707597
Cost after epoch 24: 1.693243
Cost after epoch 25: 1.676512
Cost after epoch 26: 1.660060
Cost after epoch 27: 1.639099
Cost after epoch 28: 1.623904
Cost after epoch 29: 1.605115
Cost after epoch 30: 1.579848
Cost after epoch 31: 1.563470
Cost after epoch 32: 1.538649
Cost after epoch 33: